In [58]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import requests

# Extracting every odi world cup match for later predictions

In [55]:
html_content = requests.get("https://stats.espncricinfo.com/ci/engine/records/team/match_results_season.html?id=12;type=trophy")
soup = BeautifulSoup(html_content.content,"html.parser")
table = soup.find("table",class_ = "recordsTable")
anchor = table.find_all("a")
yearwise_links = ["https://stats.espncricinfo.com" + a['href'] for a in anchor]

In [73]:
def scrap_world_cup_matches_info(links_list):
    matches_df = pd.DataFrame()
    for link in links_list:
        html_content = requests.get(link)
        soup = BeautifulSoup(html_content.content,"html.parser")
        table = soup.find("table",class_ = "engineTable")
        columns = []
        for th in table.find_all("th"):
            columns.append(th.text.strip())
        data = []
        for tr in table.find("tbody").find_all("tr"):
            row = []
            for td in tr.find_all("td"):
                row.append(td.text.strip())
            data.append(row)
        columns = ['Team 1','Team 2','Winner','Margin','Ground','Match Date','Match id']
        for row in data:
            match = row[0] + " vs " + row[1]
            row.insert(0, match)
            matches_df = matches_df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
    return matches_df

In [74]:
matches_df = scrap_world_cup_matches_info(yearwise_links)

In [79]:
matches_df.to_csv("world_cup_matches_data.csv",index=False)